In [2]:
from gurobipy import *

In [3]:
try:

    # Create a new model
    m = Model("mip1")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    m.optimize()

    for v in m.getVars():
        print(v.varName, v.x)

    print('Obj:', m.objVal)

except GurobiError:
    print('Error reported')

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
('x', 1.0)
('y', 0.0)
('z', 1.0)
('Obj:', 3.0)


In [4]:

# Model data

commodities = ['Pencils', 'Pens']
nodes = ['Detroit', 'Denver', 'Boston', 'New York', 'Seattle']

arcs, capacity = multidict({
  ('Detroit', 'Boston'):   100,
  ('Detroit', 'New York'):  80,
  ('Detroit', 'Seattle'):  120,
  ('Denver',  'Boston'):   120,
  ('Denver',  'New York'): 120,
  ('Denver',  'Seattle'):  120 })

cost = {
  ('Pencils', 'Detroit', 'Boston'):   10,
  ('Pencils', 'Detroit', 'New York'): 20,
  ('Pencils', 'Detroit', 'Seattle'):  60,
  ('Pencils', 'Denver',  'Boston'):   40,
  ('Pencils', 'Denver',  'New York'): 40,
  ('Pencils', 'Denver',  'Seattle'):  30,
  ('Pens',    'Detroit', 'Boston'):   20,
  ('Pens',    'Detroit', 'New York'): 20,
  ('Pens',    'Detroit', 'Seattle'):  80,
  ('Pens',    'Denver',  'Boston'):   60,
  ('Pens',    'Denver',  'New York'): 70,
  ('Pens',    'Denver',  'Seattle'):  30 }

inflow = {
  ('Pencils', 'Detroit'):   50,
  ('Pencils', 'Denver'):    60,
  ('Pencils', 'Boston'):   -50,
  ('Pencils', 'New York'): -50,
  ('Pencils', 'Seattle'):  -10,
  ('Pens',    'Detroit'):   60,
  ('Pens',    'Denver'):    40,
  ('Pens',    'Boston'):   -40,
  ('Pens',    'New York'): -30,
  ('Pens',    'Seattle'):  -30 }

# Create optimization model
m = Model('netflow')

# Create variables
flow = m.addVars(commodities, arcs, obj=cost, name="flow")

# Arc capacity constraints
m.addConstrs(
    (flow.sum('*',i,j) <= capacity[i,j] for i,j in arcs), "cap")


# Flow conservation constraints
m.addConstrs(
    (flow.sum(h,'*',j) + inflow[h,j] == flow.sum(h,j,'*')
    for h in commodities for j in nodes), "node")

# Compute optimal solution
m.optimize()

# Print solution
if m.status == GRB.Status.OPTIMAL:
    solution = m.getAttr('x', flow)
    for h in commodities:
        print('\nOptimal flows for %s:' % h)
        for i,j in arcs:
            if solution[h,i,j] > 0:
                print('%s -> %s: %g' % (i, j, solution[h,i,j]))

Optimize a model with 16 rows, 12 columns and 36 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 16 rows and 12 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds
Optimal objective  5.500000000e+03

Optimal flows for Pencils:
Denver -> Seattle: 10
Denver -> New York: 50
Detroit -> Boston: 50

Optimal flows for Pens:
Denver -> Seattle: 30
Detroit -> New York: 30
Detroit -> Boston: 30
Denver -> Boston: 10


# Simplified Sengupta Paper 

In [ ]:
# simple one attacker version of the Sengupta paper example

# defender_reward[i][j] is reward for attacker action i on defender configuration j
defender_rewards = [
    [0, -5],
    [-5, 0]
]

m = Model('test')

x = []
x.append(m.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name='x0'))
x.append(m.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name='x1'))

n=[]
# whether attacker plays action 0 or action 1
for j in range(2):
    n.append(m.addVar(vtype=GRB.BINARY, name="n_{0}".format(j)))
    
v = m.addVar(vtype=GRB.CONTINUOUS, name = 'v0')

# Set objective
m.setObjective(sum(defender_rewards[a][c] * x[c] * n[a] \
                   for a in [0,1] \
                   for c in [0,1]), GRB.MAXIMIZE)

m.addConstr(sum(x) == 1, 'defender_strat')
m.addConstr(sum(n) == 1, 'pure_attacker')
# NOTE: plus defender rewards instead of minus defender rewards because we assume
# this is a zero sum game 
m.addConstrs((0 <= v + sum(defender_rewards[a][c] * x[c] for c in range(2)) for a in range(2)), 'attackerstrat1')
m.addConstrs((v + sum(defender_rewards[a][c] * x[c] for c in range(2)) <= (1 - n[a]) * 99999 for a in range(2)),
            'attackerstrat2')

m.optimize()
         
# m.addConstr(x + 2 * y + 3 * z <= 4, "c0")



for var in m.getVars():
    print(var.varName, var.x)
print('Obj:', m.objVal)

In [11]:
# skills
_, attacker_skill, attacker_skill_level = {
    # name  tech. skill
    'sk' : ('php', 4),
    'sk' : ('mysql', 4)
}
# attack vars
tech_attacks = {
    # tech.     attack          ES. IS
    'python': ('python_attack1', 1, 5),
    'python': ('python_attack2', 8, 10),
    'php' : ('php_attack1', 2, 4),
    'php' : ('php_attack2', 3, 7),
    'php' : ('php_attack3', 6, 8),
    'mysql' : ('mysql_attack1', 8, 10)
}